In [2]:
import pandas as pd
import requests
import json
import numpy as np
import time

# Setting  up

In [4]:
# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth('9wXh5oa4cfW07_eUn5Hu3A', 'm3GnhaEvbM5LGCWX3BMghLCatOLN3g')
# setup our header info, which gives reddit a brief description of our app
headers = {'User-Agent': 'babi_research/0.0.1'}
# here we pass our login method (password), username, and password
data = {'grant_type': 'password',
        'username': 'softyarn',
        'password': '1532679!Qa1',
        'User-Agent':'babi_research/0.0.1',
        'refresh_token': '167791397396-1cjKrPIOvr4ct1EFwMm57hY1vi7baw'
        }

In [5]:
# send our request for an OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
# convert response to JSON and pull access_token value
res.json()

{'access_token': '167791397396-J73cjzVYRRXtPw_QmoBB3LG3F3tdNQ',
 'token_type': 'bearer',
 'expires_in': 86400,
 'scope': '*'}

In [6]:
# convert response to JSON and pull access_token value
token = res.json()['access_token']
# add authorization to our headers dictionary
headers['Authorization'] = 'bearer {}'.format(token)
# while the token is valid (~2 hours) we just add headers=headers to our requests. End point: GET /api/v1/me
requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

{'is_employee': False,
 'seen_layout_switch': False,
 'has_visited_new_profile': False,
 'pref_no_profanity': True,
 'has_external_account': False,
 'pref_geopopular': '',
 'seen_redesign_modal': False,
 'pref_show_trending': True,
 'subreddit': {'default_set': False,
  'user_is_contributor': False,
  'banner_img': '',
  'restrict_posting': True,
  'user_is_banned': False,
  'free_form_reports': True,
  'community_icon': None,
  'show_media': True,
  'icon_color': '',
  'user_is_muted': None,
  'display_name': 'u_softyarn',
  'header_img': None,
  'title': '',
  'coins': 0,
  'previous_names': [],
  'over_18': False,
  'icon_size': [256, 256],
  'primary_color': '',
  'icon_img': 'https://styles.redditmedia.com/t5_3ef3ey/styles/profileIcon_snoo86ebddf0-4659-47c8-9ad6-b6ca0588943d-headshot-f.png?width=256&amp;height=256&amp;crop=256:256,smart&amp;s=86680ec95737b7ce10db712e2889672a4de8bbf1',
  'description': '',
  'allowed_media_in_comments': [],
  'submit_link_label': '',
  'header_size

# Miscellanous scraping

In [32]:
#Get moderators of feminisms
fds_mod= requests.get('https://oauth.reddit.com/r/femaledatingstrategy/about/moderators', headers=headers).json()

In [33]:
fds_mod_df = pd.DataFrame(fds_mod["data"]["children"])
fds_mod_df

,name,author_flair_text,mod_permissions,date,rel_id,id,author_flair_css_class
0,rainisthelife,Ruthless Strategist,[all],1.551256e+09,rb_175c48g,t2_w4w0qu,None
1,TheMarbleSlab,Ruthless Strategist,[all],1.564346e+09,rb_1fpm08m,t2_22udesmu,None
2,TheOGJammies,Ruthless Strategist,[all],1.566522e+09,rb_1h2vrgy,t2_43ex46gt,None
3,AverageToHot,Ruthless Strategist,[all],1.566538e+09,rb_1h398z4,t2_3u7axe0z,None
4,throwawayy92838383,Ruthless Strategist,[all],1.569776e+09,rb_1j0legb,t2_4f0yk6fe,None
5,Liz_Lemondrop,Ruthless Strategist,[all],1.572169e+09,rb_1kflrcs,t2_4vwzr8nf,None
6,Saferbot,Ruthless Strategist,[all],1.572235e+09,rb_1kh6osb,t2_6ladm,None
7,CoolMelonade,Ruthless Strategist,[all],1.572289e+09,rb_1kia3cy,t2_4w9tmtae,None
8,makingballoons,None,"[posts, access, mail, flair]",1.572364e+09,rb_1kjwe6i,t2_ivbk9wq,None
9,NecessaryCook,Ruthless Strategist,[all],1.572386e+09,rb_1kkgl98,t2_22hgl225,None


In [9]:
fds_mod_df.iloc[6,]

name                                 Saferbot
author_flair_text         Ruthless Strategist
mod_permissions                         [all]
date                             1572234835.0
rel_id                             rb_1kh6osb
id                                   t2_6ladm
author_flair_css_class                   None
Name: 6, dtype: object

# Function

In [16]:
def clean_data(source_dir, comment_bool):
    df = pd.read_csv(source_dir)
    df = df[["id","body",'author']]
    if comment_bool == True:
        df['id'] ='t1_' + df['id'].astype(str)
    else:
        df['id'] ='t3_' + df['id'].astype(str)
    nomod = df.loc[(df['author'] != "AutoModerator") & (df['author'] != '[deleted]')]
    nomod["note"] = np.where(nomod["body"] == "[removed]", "removed", np.where(nomod["body"] == "[deleted]", "deleted", "text"))
    return nomod

In [17]:
def clean_df(df, comment_bool):
    df = df[["id","body",'author']]
    if comment_bool == True:
        df['id'] ='t1_' + df['id'].astype(str)
    else:
        df['id'] ='t3_' + df['id'].astype(str)
    nomod = df.loc[(df['author'] != "AutoModerator") & (df['author'] != '[deleted]')]
    nomod["note"] = np.where(nomod["body"] == "[removed]", "removed", np.where(nomod["body"] == "[deleted]", "deleted", "text"))
    return nomod

In [41]:
def get_comments(df, start_from=0) :
    tic = time.time()
    global res_df
    res_df = []
    i = 0
    if start_from == None:
        N = 0
    else:
        N = start_from
    while i < len(df["id"]):
        j = min(len(df["id"]), i + 100)
        x = ",".join(df["id"][i:j])
        comments = requests.get('https://oauth.reddit.com/api/info',headers=headers,params={'id': x}).json()
        try:
            for k in range(j-i):
                comment_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
                res_df.append(comment_df)
        except IndexError:
            for k in range(len(comments.get("data").get("children"))):
                comment_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
                res_df.append(comment_df)
        i += 100
        if i%200000 == 0:
            res_out = pd.concat(res_df)
            res_out.to_csv('reladvice_comments_' + str((i+start_from)/200000) + '.csv', encoding = 'utf_8_sig')
            res = []
        if i%300 == 0:
            time.sleep(3)
            #now = time.localtime(time.time())
            print('Time elapsed: ', round(time.time() - tic),' secs, i = ',i)
            #print(time.strftime("Time now: %HH:%M:%S",now),f', i = {i}')
    return pd.concat(res_df)

In [27]:
def get_comments_simple(df):
    res_df = pd.DataFrame()
    i = 0
    while i < len(df["id"]):
        x = ",".join(df["id"][i:i+20])
        comments = requests.get('https://oauth.reddit.com/api/info',headers=headers,params={'names': x}).json()
        for k in range(100):
            comments_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
            res_df.append(submission_df)
        if i%300 == 0:
            time.sleep(5)
        i += 20  
    return res_df

In [ ]:
def get_submissions(df):
    res_df = pd.DataFrame()
    i = 0
    while i < len(df["id"]):
        x = ",".join(df["id"][i:i+20])
        submissions = requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names': x}).json()
        for k in range(20):
            submission_df = pd.DataFrame.from_dict(comments.get("data").get("children")[k].get("data"), orient = "index").transpose()
            res_df.append(submission_df)
        if i%300 == 0:
            time.sleep(5)
        i += 20  
    return res_df  

# Scraping

# r/femaledatingstrategy

In [ ]:
fds = clean_data("E:/gihub-data/redditbots/fds/fds_comments.csv", comment_bool= True)

In [ ]:
fds_res = get_comments(fds)

In [ ]:
fds_res.to_csv('fds_comment_retrieved.csv')

In [ ]:
requests.get('https://oauth.reddit.com/by_id/names',headers=headers,params={'names': 't3_cipdyg'}).json()

In [ ]:
requests.get('https://oauth.reddit.com/user/')

## r/AskReddit

In [ ]:
askreddit = clean_data("E:/gihub-data/redditbots/control-fds/askreddit.csv", comment_bool= True)

In [ ]:
askreddit_res = get_comments(askreddit)

In [ ]:
askreddit_res.to_csv('E:/gihub-data/redditbots/control-fds/askreddit_comment_retrieved.csv')

# PurplePillDebate

In [ ]:
purplepill = clean_data("E:/gihub-data/redditbots/control-fds/purple.csv", comment_bool= True)

In [ ]:
purple_res = get_comments(purplepill)

In [ ]:
purple_res.to_csv('E:/gihub-data/redditbots/control-fds/purple_comment_retrieved.csv')

# r/AmItheAsshole

undone

In [12]:
AITA = clean_data("E:/gihub-data/redditbots/control-fds/AITA.csv", comment_bool= True)

NameError: name 'clean_data' is not defined

In [12]:
AITA_res = get_comments(AITA, start_from=1800000)

Time elapsed:  5  secs, i =  300
Time elapsed:  10  secs, i =  600


# r/unpopularopinion

done

In [18]:
unpop_df = pd.read_csv("E:/gihub-data/redditbots/control-fds/unpopularopinion.csv")

In [19]:
unpop_1 = clean_df(unpop_df, comment_bool=True)

C:\Users\nguye\AppData\Local\Temp\ipykernel_32332\332450768.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] ='t1_' + df['id'].astype(str)
C:\Users\nguye\AppData\Local\Temp\ipykernel_32332\332450768.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nomod["note"] = np.where(nomod["body"] == "[removed]", "removed", np.where(nomod["body"] == "[deleted]", "deleted", "text"))


In [ ]:
unpop_res = get_comments(unpop_1, start_from = 1200000) #start_from 209100

# r/gendercritical

banned

In [14]:
gender_df = pd.read_csv("E:/gihub-data/redditbots/control-fds/gendercritical.csv")

In [18]:
gender_1 = clean_df(gender_df, comment_bool=True)

C:\Users\nguye\AppData\Local\Temp\ipykernel_20376\332450768.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] ='t1_' + df['id'].astype(str)
C:\Users\nguye\AppData\Local\Temp\ipykernel_20376\332450768.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nomod["note"] = np.where(nomod["body"] == "[removed]", "removed", np.where(nomod["body"] == "[deleted]", "deleted", "text"))


# r/relationshipadvice

In [42]:
relad_df = pd.read_csv("E:/gihub-data/redditbots/control-fds/relationshipadvice.csv")

C:\Users\nguye\AppData\Local\Temp\ipykernel_20376\801646319.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  relad_df = pd.read_csv("E:/gihub-data/redditbots/control-fds/relationshipadvice.csv")


In [43]:
relad_df1 = clean_df(relad_df, comment_bool=True)

C:\Users\nguye\AppData\Local\Temp\ipykernel_20376\332450768.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] ='t1_' + df['id'].astype(str)
C:\Users\nguye\AppData\Local\Temp\ipykernel_20376\332450768.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nomod["note"] = np.where(nomod["body"] == "[removed]", "removed", np.where(nomod["body"] == "[deleted]", "deleted", "text"))


In [44]:
relad_res = get_comments(relad_df1, start_from = 0)

Time elapsed:  8  secs, i =  300
Time elapsed:  15  secs, i =  600
Time elapsed:  23  secs, i =  900
Time elapsed:  30  secs, i =  1200
Time elapsed:  38  secs, i =  1500
Time elapsed:  46  secs, i =  1800
Time elapsed:  53  secs, i =  2100
Time elapsed:  60  secs, i =  2400
Time elapsed:  67  secs, i =  2700
Time elapsed:  74  secs, i =  3000
Time elapsed:  82  secs, i =  3300
Time elapsed:  89  secs, i =  3600
Time elapsed:  96  secs, i =  3900
Time elapsed:  104  secs, i =  4200
Time elapsed:  111  secs, i =  4500
Time elapsed:  119  secs, i =  4800
Time elapsed:  126  secs, i =  5100
Time elapsed:  133  secs, i =  5400
Time elapsed:  140  secs, i =  5700
Time elapsed:  148  secs, i =  6000
Time elapsed:  154  secs, i =  6300
Time elapsed:  161  secs, i =  6600
Time elapsed:  168  secs, i =  6900
Time elapsed:  175  secs, i =  7200
Time elapsed:  182  secs, i =  7500
Time elapsed:  189  secs, i =  7800
Time elapsed:  197  secs, i =  8100
Time elapsed:  204  secs, i =  8400
Time elap

KeyboardInterrupt: 